# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [1]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [2]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [3]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [4]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from barcode_runs_subset.csv
Here are the first few entries in the Illumina runs:


,library,sample,date,experiment,antibody,concentration,sort_bin,HutchBase,experiment_type,number_cells,frac_escape,R1
0,lib1,delta_21-26-none-0-ref,211119,delta_21-26,none,0.0,ref,delta_21-26_lib1_ref,ab_selection,NaN,NaN,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_21-26_lib1_ref*R1*.fastq.gz; /shared/ngs/illumina/agreaney/211208_D00300_1390_AHNKGMBCX3/Unaligned/Project_agreaney/delta_21-26_lib1_ref*R1*.fastq.gz
1,lib2,delta_21-26-none-0-ref,211119,delta_21-26,none,0.0,ref,delta_21-26_lib2_ref,ab_selection,NaN,NaN,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_21-26_lib2_ref*R1*.fastq.gz; /shared/ngs/illumina/agreaney/211208_D00300_1390_AHNKGMBCX3/Unaligned/Project_agreaney/delta_21-26_lib2_ref*R1*.fastq.gz
2,lib1,delta_17-20-none-0-ref,211112,delta_17-20,none,0.0,ref,delta_17-20_lib1_ref,ab_selection,NaN,NaN,/shared/ngs/illumina/agreaney/211117_D00300_1372_BHNH7WBCX3/Unaligned/Project_agreaney/delta_17-20_lib1_ref*R1*.fastq.gz
3,lib2,delta_17-20-none-0-ref,211112,delta_17-20,none,0.0,ref,delta_17-20_lib2_ref,ab_selection,NaN,NaN,/shared/ngs/illumina/agreaney/211117_D00300_1372_BHNH7WBCX3/Unaligned/Project_agreaney/delta_17-20_lib2_ref*R1*.fastq.gz
4,lib1,delta_27-34-none-0-ref,211122,delta_27-34,none,0.0,ref,delta_27-34_lib1_ref,ab_selection,NaN,NaN,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_27-34_lib1_ref*R1*.fastq.gz; /shared/ngs/illumina/agreaney/211208_D00300_1390_AHNKGMBCX3/Unaligned/Project_agreaney/delta_27-34_lib1_ref*R1*.fastq.gz


Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [5]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + '_' + x['sample'],
        title=lambda x: 'SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for ' + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename
0,SAMN26317640,lib1_delta_21-26-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_21-26-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_21-26_lib1_ref-2_S2_R1_001.fastq.gz,delta_21-26_lib1_ref-2_S2_R1_001.fastq.gz
1,SAMN26317640,lib1_delta_21-26-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_21-26-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_21-26_lib1_ref-4_S4_R1_001.fastq.gz,delta_21-26_lib1_ref-4_S4_R1_001.fastq.gz
2,SAMN26317640,lib1_delta_21-26-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_21-26-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_21-26_lib1_ref-1_S1_R1_001.fastq.gz,delta_21-26_lib1_ref-1_S1_R1_001.fastq.gz
3,SAMN26317640,lib1_delta_21-26-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_21-26-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_21-26_lib1_ref-3_S3_R1_001.fastq.gz,delta_21-26_lib1_ref-3_S3_R1_001.fastq.gz
4,SAMN26317640,lib2_delta_21-26-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_21-26-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/agreaney/211130_VH00699_22_AAANYJTM5/Unaligned/Project_agreaney/delta_21-26_lib2_ref-1_S5_R1_001.fastq.gz,delta_21-26_lib2_ref-1_S5_R1_001.fastq.gz


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [6]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,biosample_accession,library_ID,n_files
0,SAMN26317640,lib1_delta_17-20-none-0-ref,4
1,SAMN26317640,lib1_delta_27-34-none-0-ref,4
2,SAMN26317640,lib2_delta_35-40-none-0-ref,4
3,SAMN26317640,lib2_delta_27-34-none-0-ref,4
4,SAMN26317640,lib2_delta_21-26-none-0-ref,4
5,SAMN26317640,lib1_delta_35-40-none-0-ref,4
6,SAMN26317640,lib2_delta_17-20-none-0-ref,4
7,SAMN26317640,lib1_delta_21-26-none-0-ref,4
8,SAMN26317640,lib1_delta_22-273C-500-abneg,1
9,SAMN26317640,lib2_delta_29-Delta_4-350-abneg,1


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [7]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2,filename3,filename4
0,SAMN26317640,lib1_delta_21-26-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_21-26-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,delta_21-26_lib1_ref-2_S2_R1_001.fastq.gz,delta_21-26_lib1_ref-4_S4_R1_001.fastq.gz,delta_21-26_lib1_ref-1_S1_R1_001.fastq.gz,delta_21-26_lib1_ref-3_S3_R1_001.fastq.gz
1,SAMN26317640,lib2_delta_21-26-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_21-26-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,delta_21-26_lib2_ref-1_S5_R1_001.fastq.gz,delta_21-26_lib2_ref-3_S7_R1_001.fastq.gz,delta_21-26_lib2_ref-2_S6_R1_001.fastq.gz,delta_21-26_lib2_ref-4_S8_R1_001.fastq.gz
2,SAMN26317640,lib1_delta_17-20-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_17-20-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,delta_17-20_lib1_ref-2_S10_R1_001.fastq.gz,delta_17-20_lib1_ref-3_S11_R1_001.fastq.gz,delta_17-20_lib1_ref-4_S12_R1_001.fastq.gz,delta_17-20_lib1_ref-1_S9_R1_001.fastq.gz
3,SAMN26317640,lib2_delta_17-20-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_17-20-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,delta_17-20_lib2_ref-1_S13_R1_001.fastq.gz,delta_17-20_lib2_ref-3_S15_R1_001.fastq.gz,delta_17-20_lib2_ref-2_S14_R1_001.fastq.gz,delta_17-20_lib2_ref-4_S16_R1_001.fastq.gz
4,SAMN26317640,lib1_delta_27-34-none-0-ref,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for delta_27-34-none-0-ref,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,delta_27-34_lib1_ref-4_S28_R1_001.fastq.gz,delta_27-34_lib1_ref-1_S25_R1_001.fastq.gz,delta_27-34_lib1_ref-3_S27_R1_001.fastq.gz,delta_27-34_lib1_ref-2_S26_R1_001.fastq.gz


Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [8]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [9]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
